    Imports

In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

    Funccion a grosso modo

In [60]:
url = 'https://www.vogue.com/fashion-shows/latest-shows'
response = requests.get(url)
designer_dict = {}

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Contenido HTML de la página web
    html_content = response.text
    print(f"Pagina obtenida. \nhead de la página: \n{html_content[:30]}")
else:
    print("Error al obtener la página:", response.status_code)
    
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
# Encontrar todos los elementos <a> en la página
enlaces = soup.find_all('a')
year = "2024"
designer_dict = {}
# df = pd.DataFrame(columns=['designer', 'img'])

# Imprimir los enlaces encontrados
for enlace in enlaces:
    # print(f"Descargando enlace: {enlace.get('href')}")
    enlace = enlace.get('href')
    if year in enlace:
        designer_page = "https://www.vogue.com"+ enlace
        designer_name = enlace.split('/')[-1]
        print(f"Descargando enlace de {designer_name}: {designer_page}")
        response = requests.get(designer_page)
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            # sacar la pagina de ver las imagenes
            boton = soup.find_all(class_="RunwayShowPageGalleryCta-fmTQJF FtdcL")
            # sacar el link de la pagina de imagenes
            link = boton[0].find('a').get('href')
            # driver = webdriver.Chrome()
            driver = webdriver.Chrome(options=chrome_options)
            collection_page = f"https://www.vogue.com{link}"
            driver.get(collection_page)
            try:
                terms_button = driver.find_element('xpath', '//*[@id="onetrust-accept-btn-handler"]')
                terms_button.click() if terms_button else None
            except NoSuchElementException:
                print("No se encontró el botón de aceptar términos. Continuando sin hacer clic.")
            time.sleep(3)
            try:
                ad_button = driver.find_element('xpath', '//*[@id="app-root"]/div/div/div[6]/div[1]/div/button')
                ad_button.click() if ad_button else None
            except NoSuchElementException:
                print("No se encontró el botón de publicidad. Continuando sin hacer clic.")
            image_count = 0
            previous_page_source = None
            while True:
                try:
                    # button = driver.find_element('xpath', '//*[@id="main-content"]/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div[3]')
                    button = WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.XPATH, 
                                '//*[@id="main-content"]/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div[3]')))
                    # print(button)
                    # if button:
                    if button.is_enabled():
                        image_count += 1
                        button.click()
                        time.sleep(1)
                        img = driver.find_element('xpath', '//*[@id="main-content"]/div/div[1]/div[2]/div/div/ul/li/div[1]/div/span/picture/img')
                        img_url = img.get_attribute('src')
                        current_page_source = driver.page_source
                        print(f"Descargando imagen {image_count} de {designer_name}: {img_url}")
                        if designer_name not in designer_dict:
                            designer_dict[designer_name] = [img_url]
                        else:
                            designer_dict[designer_name].append(img_url)
                        
                        if current_page_source == previous_page_source:
                            print("No hay más imágenes disponibles. Se alcanzó el final de la colección.")
                            driver.quit()
                            break
                        else:
                            previous_page_source = current_page_source
                            
                    else:
                        print("El botón está deshabilitado. Se alcanzó el final de la colección.")
                        break
                            
                except Exception as e:
                    print("Error:", e)
                    break  # Salir del bucle si hay un error o no hay más páginas

                    
               
 



driver.quit()

Pagina obtenida. 
head de la página: 
<!DOCTYPE html><html lang="en-
Descargando enlace de rag-bone: https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/rag-bone
Descargando imagen 1 de rag-bone: https://assets.vogue.com/photos/65c249a236b4e0759ccfb5f9/master/w_2560%2Cc_limit/00002-rag-and-bone-fall-2024-ready-to-wear-credit-brand.jpeg
Descargando imagen 2 de rag-bone: https://assets.vogue.com/photos/65c249a00140a4bcc056b573/master/w_2560%2Cc_limit/00003-rag-and-bone-fall-2024-ready-to-wear-credit-brand.jpeg
Descargando imagen 3 de rag-bone: https://assets.vogue.com/photos/65c249a4ecd62385787a69b7/master/w_2560%2Cc_limit/00004-rag-and-bone-fall-2024-ready-to-wear-credit-brand.jpeg
Descargando imagen 4 de rag-bone: https://assets.vogue.com/photos/65c249a9d9c257479ca0f398/master/w_2560%2Cc_limit/00005-rag-and-bone-fall-2024-ready-to-wear-credit-brand.jpeg
Descargando imagen 5 de rag-bone: https://assets.vogue.com/photos/65c249a70140a4bcc056b575/master/w_2560%2Cc_limit/00006-rag-a

In [63]:
# Lista para almacenar los datos
data = []

# Recorrer el diccionario y agregar los datos a la lista
for designer, urls in designer_dict.items():
    for url in urls:
        data.append({'designer': designer, 'url': url})

# Crear el DataFrame a partir de la lista
df = pd.DataFrame(data)


In [62]:
df.to_csv('vogue.csv', index=False)

    Main page

In [32]:
url = 'https://www.vogue.com/fashion-shows/latest-shows'
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Contenido HTML de la página web
    html_content = response.text
    print(f"Pagina obtenida. \nhead de la página: \n{html_content[:30]}")
else:
    print("Error al obtener la página:", response.status_code)
    
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

Pagina obtenida. 
head de la página: 
<!DOCTYPE html><html lang="en-


    Encontrar pagina de los diseñadores (parte dict y imagenes WIP)

In [29]:
def get_enlaces(enlaces, year:int):
    year = str(year)
    for enlace in enlaces:
        # print(f"Descargando enlace: {enlace.get('href')}")
        enlace = enlace.get('href')
        if year in enlace:
            designer_page = "https://www.vogue.com"+ enlace
            designer_name = enlace.split('/')[-1]
            print(f"Descargando enlace de {designer_name}: {designer_page}")
            return designer_page, designer_name



In [30]:
get_enlaces(soup.find_all('a'), 2021)

In [4]:
# Encontrar todos los elementos <a> en la página
enlaces = soup.find_all('a')
year = "2024"
designer_dict = {}
# df = pd.DataFrame(columns=['designer', 'img'])

# Imprimir los enlaces encontrados
for enlace in enlaces:
    # print(f"Descargando enlace: {enlace.get('href')}")
    enlace = enlace.get('href')
    if year in enlace:
        designer_page = "https://www.vogue.com"+ enlace
        designer_name = enlace.split('/')[-1]
        print(f"Descargando enlace de {designer_name}: {designer_page}")
        response = requests.get(designer_page)
        if response.status_code == 200:
            html_content = response.text
        #     soup = BeautifulSoup(html_content, 'html.parser')
            # print(soup.find_all('img'))
            for img in soup.find_all('img'):
                if "SummaryItemSimpleHed" not in img.get('alt'):
                    continue
                # print(img.get('src'))
                if img.get('src') is not None and img.get('src').endswith('.jpg'):
                    if designer_name not in designer_dict:
                        designer_dict[designer_name] = [img.get('src')]
                    else:
                        designer_dict[designer_name].append(img.get('src'))
        else:
            print("Error al obtener la página:", response.status_code)

Descargando enlace de vince: https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/vince
Descargando enlace de sara-battaglia: https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia
Descargando enlace de moncler-grenoble: https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/moncler-grenoble
Descargando enlace de sunflower: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/sunflower
Descargando enlace de soulland: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/soulland
Descargando enlace de helmstedt: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/helmstedt
Descargando enlace de caro-editions: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/caro-editions
Descargando enlace de mark-kenly-domino-tan: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/mark-kenly-domino-tan
Descargando enlace de paolina-russo: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/paolina-russo
Descargando enlace de baum-und-pferdg

    DF con diferente tamaño vertical (teniendo en cuenta el key con mas len()del  dict y los demás a None)

In [5]:
max_len = max(len(lst) for lst in designer_dict.values())
for key in designer_dict:
    len_diff = max_len - len(designer_dict[key])
    if len_diff > 0:
        designer_dict[key] += [None] * len_diff

df = pd.DataFrame(designer_dict)

ValueError: max() arg is an empty sequence

    Encontrar página de all pictures en cada diseñador

In [32]:
designer_page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia"

response = requests.get(designer_page)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    # sacar la pagina de ver las imagenes
    boton = soup.find_all(class_="RunwayShowPageGalleryCta-fmTQJF FtdcL")
    # sacar el link de la pagina de imagenes
    link = boton[0].find('a').get('href')


In [33]:
link

'/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection'

    Encontrar imagenes en allpics (ESTOY ATASCADO) // en esta parte siempre recoge la misma imagen aunque se itere sobre varias páginas

In [46]:
driver = webdriver.Chrome()
collection_page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection"
driver.get(collection_page)
terms_button = driver.find_element('xpath', '//*[@id="onetrust-accept-btn-handler"]')
terms_button.click()
button = driver.find_element('xpath', '//*[@id="main-content"]/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div[3]')
while button:
    button.click()
    # time.sleep(2)
    img = driver.find_element('xpath', '//*[@id="main-content"]/div/div[1]/div[2]/div/div/ul/li/div[1]/div/span/picture/img')
    print(img.get_attribute('src'))
       

button: <selenium.webdriver.remote.webelement.WebElement (session="488b808613b2fe665e805f22037c780d", element="395FC05C960592C053AB0E5DAD6513FD_element_33")>
https://assets.vogue.com/photos/65ba546a5f8958b63b8d488d/master/w_2560%2Cc_limit/00002-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://assets.vogue.com/photos/65ba546a4f005afdef36c860/master/w_2560%2Cc_limit/00003-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://assets.vogue.com/photos/65ba546dfa392b10eb9f1394/master/w_2560%2Cc_limit/00004-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://assets.vogue.com/photos/65ba546d6e24549428bffc40/master/w_2560%2Cc_limit/00005-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://assets.vogue.com/photos/65ba546d6e24549428bffc41/master/w_2560%2Cc_limit/00006-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://assets.vogue.com/photos/65ba546f14494a9f83aa8cbf/master/w_2560%2Cc_limit/00007-sara-battaglia-fall-2024-ready-to-wear-c

KeyboardInterrupt: 

In [9]:
collection_page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection"
# range de 10 como ejemplo
for i in range(10):
    collection_page_iter = f"{collection_page}#{str(i+1)}"
    # print(collection_page_iter, "\n")
    response = requests.get(f"{collection_page}#{str(i+1)}")
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        for img in soup.find_all(class_="ResponsiveAssetWrapper-gdcDmm eeYyjw"):
            img = img.find('img')
            src = img.get('src')
            if src is not None and src.endswith('.jpg'):
                # print(src)
                None
        time.sleep(2)  # Esperar un segundo antes de la siguiente solicitud

    else:
        print("Error al obtener la página:", response.status_code)



KeyboardInterrupt: 

In [11]:

# Crear una nueva instancia del navegador Chrome
page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#3"

# hacer que sea headless


driver = webdriver.Chrome()
driver.get(page)
time.sleep(2)
# # Encontrar el elemento con el xpath
# img = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[2]/div/div/div[1]/div/div[1]/div/div/div/div[1]/div/div/div/div/div[1]/div/img')
img = driver.find_element('xpath', '//*[@id="main-content"]/div/div[1]/div[2]/div/div/ul/li/div[1]/div/span/picture/img')
# # Imprimir el src del elemento
print(img.get_attribute('src'))

https://assets.vogue.com/photos/65ba546a4f005afdef36c860/master/w_2560%2Cc_limit/00003-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg


In [8]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#3"

soup = BeautifulSoup(page.content, 'html.parser')
body = soup.find("body")

dom = etree.HTML(str(body)) # Parse the HTML content of the page
xpath_str = '//*[@id="main-content"]/div/div[1]/div[2]/div/div/ul/li/div[1]/div/span/picture/img' # The XPath expression for the blog's title
print(dom.xpath(xpath_str)[0].text)

AttributeError: 'str' object has no attribute 'content'